Here I will try and prove that my model isn't working. First, I will run my optimizer 3 times to see if I get different results.

Now I will compare how the mode matching compares to actually tracing the particle. I will also inspect these swarms to see that they make sense.




In [1]:
from latticeKnobOptimizer import LatticeOptimizer
import dill
import matplotlib.pyplot as plt
from ParticleClass import Swarm,Particle
from ParticleTracerLatticeClass import ParticleTracerLattice
import numpy as np
import time
from SwarmTracerClass import SwarmTracer
from runOptimizer import generate_Ring_Lattice,generate_Injector_Lattice
parallel=False


XTune1= [0.01081848, 0.01021685, 0.02     ,  0.01834457 ,0.30497962, 0.51553632 ,0.65781728]

rpBend,rpLens,rpLensFirst,Lm,LLens,injectorFactor,rpInjectorFactor=XTune1
XInjector=injectorFactor,rpInjectorFactor
XRing=rpLens,Lm,LLens
PTL_Ring=generate_Ring_Lattice(rpBend,rpLens,rpLensFirst,Lm,LLens,parallel=parallel)
PTL_Injector=generate_Injector_Lattice(injectorFactor,rpInjectorFactor,parallel=parallel)

XLatticeSpace= [0.622153   ,0.68697404]
XInjectorSpace= [0.23908808 ,0.11975553]




# file=open('ringFile','wb')
# dill.dump(PTL_Ring,file)
# file=open('ringFile','rb')
# PTL_Ring=dill.load(file)



optimizer=LatticeOptimizer(PTL_Ring,PTL_Injector)
# optimizer.numParticlesRing=50000
# optimizer.numParticlesInjector=1000
elementIndices=(1,7)
ringTuningBounds = [(.2, .8), (.2, .8)]
injectorTuningBounds = [(.05, .4), (.05, .4)]
optimizer.catch_Optimizer_Errors(ringTuningBounds, elementIndices, 'spacing')
optimizer.initialize_Optimizer(elementIndices,'spacing',ringTuningBounds,injectorTuningBounds)
optimizer.update_Ring_Lattice(XLatticeSpace)
optimizer.update_Injector_Lattice(XInjectorSpace)

# file=open('ringFile','wb')
# dill.dump(PTL_Ring,file)



In [8]:
#trace through injector
fastMode=True
h=5e-6
swarmTest=Swarm()
swarmTest.add_Particle()
swarmInjectorTraced = optimizer.swarmTracerInjector.trace_Swarm_Through_Lattice(
    swarmTest.quick_Copy()
    , h, 1.0, parallel=False,
    fastMode=fastMode, copySwarm=False,
    accelerated=True)
# print(swarmInjectorTraced.num_Particles())
swarmEnd = optimizer.move_Survived_Particles_In_Injector_Swarm_To_Origin(swarmInjectorTraced, copyParticles=False)
# print(swarmEnd.num_Particles())
swarmRingInitial = optimizer.swarmTracerRing.move_Swarm_To_Combiner_Output(swarmEnd, copySwarm=False,scoot=True)
# optimizer.latticeInjector.show_Lattice(swarm=swarmInjectorTraced,trueAspectRatio=False,showTraceLines=True,traceLineAlpha=.25,showMarkers=True,finalCoords=False)

In [9]:
#trace through lattice
fastMode=False
%matplotlib tk
T=1/200
swarmTraced = optimizer.swarmTracerRing.trace_Swarm_Through_Lattice(swarmRingInitial, optimizer.h, T, parallel=False,
                                                                    fastMode=fastMode, accelerated=True, copySwarm=True)
PTL_Ring.show_Lattice(swarm=swarmTraced,showMarkers=False,showTraceLines=True,traceLineAlpha=.5)
print(swarmTraced.survival_Bool())

1.0
